In [2]:
import requests
from pydantic import BaseModel, ConfigDict, Field, field_validator, ValidationError, IPvAnyAddress

In [13]:
class IPGeo(BaseModel):
    model_config = ConfigDict(extra="ignore")

    ip: IPvAnyAddress
    country: str | None = None
    country_code: str | None = Field(default=None, min_length=2, max_length=2)
    country_code3: str | None = Field(default=None, min_length=2, max_length=3)
    city: str | None = None
    region: str | None = None
    timezone: str | None = None
    organization_name: str | None = None

    @field_validator("organization_name", mode="after")
    @classmethod
    def set_uknown_to_none(cls, value: str) -> str:
        if value.casefold() == "uknown":
            return None
        return value


In [14]:
IPGeo(ip="8.9.8.8", organization_name="Uknown")

IPGeo(ip=IPv4Address('8.9.8.8'), country=None, country_code=None, country_code3=None, city=None, region=None, timezone=None, organization_name=None)

In [15]:
url_query = "https://get.geojs.io/v1/geo/{ip_address}.json"

In [16]:
url = url_query.format(ip_address="8.8.8.8")
response = requests.get(url)

In [17]:
type(response)

requests.models.Response

In [18]:
response_json = response.json()

In [19]:
print(response.json())

{'continent_code': 'NA', 'country': 'United States', 'latitude': '37.751', 'longitude': '-97.822', 'accuracy': 1000, 'ip': '8.8.8.8', 'area_code': '0', 'organization': 'AS15169 GOOGLE', 'country_code': 'US', 'timezone': 'America/Chicago', 'asn': 15169, 'organization_name': 'GOOGLE', 'country_code3': 'USA'}


In [20]:
type(response.json())

dict

In [21]:
data = IPGeo.model_validate(response.json())

In [22]:
data

IPGeo(ip=IPv4Address('8.8.8.8'), country='United States', country_code='US', country_code3='USA', city=None, region=None, timezone='America/Chicago', organization_name='GOOGLE')

In [23]:
url = url_query.format(ip_address="23.62.177.155")
response = requests.get(url)
response.raise_for_status()
data = IPGeo.model_validate(response.json())
print(data.model_dump_json(indent=2))

{
  "ip": "23.62.177.155",
  "country": "United States",
  "country_code": "US",
  "country_code3": "USA",
  "city": "El Segundo",
  "region": "California",
  "timezone": "America/Los_Angeles",
  "organization_name": "AKAMAI-AS"
}


In [24]:
url = "https://get.geojs.io/v1/ip/geo.json"
response = requests.get(url)
response.raise_for_status
data = IPGeo.model_validate(response.json())
print(data.model_dump_json(indent=2))

{
  "ip": "119.17.50.98",
  "country": "Australia",
  "country_code": "AU",
  "country_code3": "AUS",
  "city": "Melbourne",
  "region": "Victoria",
  "timezone": "Australia/Melbourne",
  "organization_name": "Broadband Solutions Pty Ltd"
}
